In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
from sklearn.metrics import accuracy_score 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adelalkhamisy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# create a dataframe with two columns "Rate" and "Comment" to make the preprocessing easy
reviewTrain = pd.read_csv("train_file.txt", sep='\t',header=None,names=['Rate','Comment'], skip_blank_lines=True, dtype = str, na_filter=False,infer_datetime_format=True, error_bad_lines=False)

/Users/adelalkhamisy/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
reviewTrain.head()

,Rate,Comment
0,-1,German filmmaker Ulli Lommel has managed a tas...
1,+1,It is an excellent thriller from Turkey which ...
2,+1,"Finally, the uncut version of ""Baby Face"" surf..."
3,+1,In this glorious telling of a weekend shared a...
4,+1,"Dog Bite Dog isn't going to be for everyone, b..."


In [4]:
reviewTest = pd.read_csv("test_file.txt", sep='\t',header=None,names=['Comment'], skip_blank_lines=True, dtype = str, na_filter=False,infer_datetime_format=True, error_bad_lines=False)

/Users/adelalkhamisy/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
"""
Function: preProcessing(corpus): 
It does many necessary preprocessing to data as follow:
1. delete anything that does not belong to [a-z] or [A-Z] from the data
2. transfer all characters to lower case
3. remove all stop words from the data except the word "not" because it is necessary information
4. apply stemming to data

Parameter: dataframe of reviewDataset 
return: list of preprocessed corpus of reviewDataset according to the above 4 steps
"""
def preProcessing(reviewDataset):
        preprocessedCorpus = []
        # iterate over ratring in datafame
        for i in range(len(reviewDataset)):
            #delete all symbols except a-z and A-Z in the Comment column
            comment = re.sub('[^a-zA-Z]', ' ', reviewDataset['Comment'][i])
            comment = comment.lower()
            comment = comment.split()
            porterstemmer = PorterStemmer() 
            sw = stopwords.words('english')
            #exclude not from stop word set
            sw.remove('not')
            # apply stemming to words in comment 
            comment = [porterstemmer.stem(word) for word in comment if not word in set(sw)]
            #join the words 
            comment = ' '.join(comment)
            #add train data to corpus
            preprocessedCorpus.append(comment)
        return preprocessedCorpus

In [6]:
#preprocess data
sentiment = reviewTrain.iloc[:,0].values
preprocessedReviewTrain = preProcessing(reviewTrain)
preprocessedReviewTest = preProcessing(reviewTest)

In [7]:
# create spase matrix using Tf-idf vectorizer for training and testing data
v = TfidfVectorizer(ngram_range=(2,5), max_features=30000)
tfidfTrain = v.fit_transform(preprocessedReviewTrain)
tfidfTest = v.transform(preprocessedReviewTest)

In [8]:
def knn(k, testTfidf, trainTfidf, x_test, y_train):
    prediction = []    
    for i, l in enumerate(x_test):
        # calculate cosine similarity with the help of cosine_similarity in sklearn 
        cs = cosine_similarity(tfidfTest[i], tfidfTrain).flatten()
        # utilizing numpy.argsort to find the K neighbors indices      
        nearestNeighborIndices = cs.argsort()[:-k:-1]
        #get the list of k nearest neighbor  from the sentiment  
        nearestNeighborList = y_train[nearestNeighborIndices]
        
        # convert nearestNeighborList from string type to int type
        integer_map=map(int, nearestNeighborList)
        nearestNeighborList = list(integer_map)
        
        ''' if sum of the nearestNeighborList is >0, then this mean that the +1 class is the majority 
        if the sum is zero, then the positive and negative are equal, but the program will classify it as positive
        if the the sum is negative, then the majority of the neighbors are "-1" 
        '''
        decision = sum(nearestNeighborList)
        
        if decision >= 0:
          prediction.append("+1")
        else:
          prediction.append("-1")
            
    return pd.DataFrame(prediction)

In [11]:
submission=knn(236,tfidfTest, tfidfTrain, preprocessedReviewTest, sentiment) 

In [12]:
np.savetxt('./submission.txt', submission,fmt='%s')